In [1010]:
from sagemaker.sklearn.estimator import SKLearn

import joblib

output_path = 's3://datalake-d1-bucket/mlchurn/Input/data/'

role = 'Sagemaker_test_role'

# Create the SKLearn Object by directing it to the aws_sklearn_main.py script
aws_sklearn = SKLearn(entry_point='main.py',
                      train_instance_type='ml.m4.xlarge',
                      role=role,
                     py_version="py3",
                      framework_version = '0.20.0' ,
                     source_dir="./")

# Train the model using by passing the path to the S3 bucket with the training data
aws_sklearn.fit({'train': 's3://datalake-d1-bucket/mlchurn/Input/data'})



#'model-dir': 's3://datalake-d1-bucket/mlchurn/Input/data'


train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-07-26 10:03:56 Starting - Starting the training job...
2021-07-26 10:04:21 Starting - Launching requested ML instancesProfilerReport-1627293835: InProgress
......
2021-07-26 10:05:26 Starting - Preparing the instances for training.........
2021-07-26 10:06:51 Downloading - Downloading input data...
2021-07-26 10:07:21 Training - Downloading the training image...
2021-07-26 10:07:53 Training - Training image download completed. Training in progress..2021-07-26 10:07:55,095 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-07-26 10:07:55,098 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-07-26 10:07:55,112 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-07-26 10:08:56,070 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-07-26 10:08:56,085 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-07-26

In [1011]:
aws_sklearn

In [1014]:
from sagemaker.predictor import csv_serializer, json_deserializer
# Deploy model
aws_sklearn_predictor = aws_sklearn.deploy(instance_type='ml.m4.xlarge',
                                           initial_instance_count=1,
                                           endpoint_name = 'insurance-churn-gradientboost-endpoint',
                                          serializer = csv_serializer)

# Print the endpoint to test in next step
print(aws_sklearn_predictor.endpoint)


# Uncomment and run to terminate the endpoint after you are finished
#aws_sklearn_predictor.delete_endpoint()

#aws_sklearn_predictor.content_type = 'text/csv'
# aws_sklearn_predictor.serializer = csv_serializer
# aws_sklearn_predictor.deserializer = None

-----------------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


insurance-churn-gradientboost-endpoint


In [1022]:
##r = '57,M,3,Unknown,Single,$120K +,Silver,49,1,2,1,34516.0,2517,31999.0,0.693,8596,102,0.672,0.073'
r = '57,M,3,Unknown,Single,$120K +,Silver,49,1,2,1,34516.0,2517,31999.0,0.693,8596,102,0.672,0.073\n51,F,3,Uneducated,Married,Less than $40K,Blue,39,3,3,0,8148.0,701,7447.0,0.8190000000000001,4124,87,0.8909999999999999,0.086'

In [1028]:
import os
import boto3
import pandas as pd
import csv

 

client = boto3.client('sagemaker-runtime')
endpoint_name = 'insurance-churn-gradientboost-endpoint'
# content_type = 'text/csv'

 

#k = open('../df_test.csv')
k = pd.read_csv("../df_test.csv")
import io
from io import StringIO
csv_file = io.StringIO()
# by default sagemaker expects comma seperated
k.to_csv(csv_file, sep=",", header=False, index=False)
my_payload_as_csv = csv_file.getvalue()

# # Make call to endpoint
response = client.invoke_endpoint(EndpointName=endpoint_name,
     Body=   my_payload_as_csv,
#     Body= r,
    ContentType='text/csv'
    )

print(response['Body'].read())

b'Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Churn|Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Churn|Non-Ch

In [1021]:
my_payload_as_csv

'51,F,3,Uneducated,Married,Less than $40K,Blue,39,3,3,0,8148.0,701,7447.0,0.8190000000000001,4124,87,0.8909999999999999,0.086\n55,F,2,Graduate,Married,Unknown,Blue,41,1,1,1,3591.0,1760,1831.0,0.507,13289,121,0.754,0.49\n53,F,1,Graduate,Married,Less than $40K,Blue,32,4,2,3,7832.0,1784,6048.0,0.711,14614,106,0.6559999999999999,0.228\n43,F,5,Uneducated,Married,$40K - $60K,Blue,36,5,3,2,2311.0,1320,991.0,0.6579999999999999,3967,71,0.69,0.5710000000000001\n56,F,1,Graduate,Single,Less than $40K,Blue,38,4,1,3,1737.0,0,1737.0,0.953,4723,78,0.625,0.0\n57,M,2,High School,Single,$60K - $80K,Blue,47,6,3,4,2248.0,0,2248.0,0.735,4439,77,0.54,0.0\n42,F,2,Uneducated,Married,$40K - $60K,Blue,29,1,2,2,1622.0,0,1622.0,0.7090000000000001,4633,67,0.634,0.0\n54,F,2,Doctorate,Married,Less than $40K,Blue,35,1,4,2,1685.0,0,1685.0,0.457,4155,85,0.667,0.0\n40,M,4,Graduate,Single,$80K - $120K,Silver,22,3,1,3,34516.0,0,34516.0,0.64,1028,26,0.3,0.0\n38,F,2,Unknown,Married,$40K - $60K,Blue,28,6,2,2,1628.0,0,1628.0,0

In [1005]:
k = pd.read_csv("../df_test.csv")
import io
from io import StringIO
csv_file = io.StringIO()
# by default sagemaker expects comma seperated
k.to_csv(csv_file, header=False, index=False)
my_payload_as_csv = csv_file.getvalue()

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False
    
def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False
    
samples = []
for row in my_payload_as_csv.split('\n'):
    list1 = []
    if row=="":
        continue
    row = str(row).replace("b'", "") 
    row = row.replace("\\n'", "") 
    row_to_list = row.split(',')
    for i in row_to_list:
        if isint(i)==True:
            list1.append(int(i))
        elif isfloat(i)==True:
            list1.append(float(i))
        else:
            list1.append(i)
    samples.append(list1)

In [1007]:
df = pd.DataFrame(samples,columns=['Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio'])
df.dtypes

Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [1008]:
df

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,51,F,3,Uneducated,Married,Less than $40K,Blue,39,3,3,0,8148.0,701,7447.0,0.819,4124,87,0.891,0.086
1,55,F,2,Graduate,Married,Unknown,Blue,41,1,1,1,3591.0,1760,1831.0,0.507,13289,121,0.754,0.490
2,53,F,1,Graduate,Married,Less than $40K,Blue,32,4,2,3,7832.0,1784,6048.0,0.711,14614,106,0.656,0.228
3,43,F,5,Uneducated,Married,$40K - $60K,Blue,36,5,3,2,2311.0,1320,991.0,0.658,3967,71,0.690,0.571
4,56,F,1,Graduate,Single,Less than $40K,Blue,38,4,1,3,1737.0,0,1737.0,0.953,4723,78,0.625,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,42,F,4,College,Married,$40K - $60K,Blue,34,1,3,2,1695.0,0,1695.0,0.453,2055,46,0.314,0.000
1995,49,F,2,Graduate,Single,Less than $40K,Blue,31,5,1,2,5456.0,951,4505.0,0.737,1096,27,0.588,0.174
1996,52,F,2,Graduate,Married,Less than $40K,Blue,43,5,2,2,8203.0,654,7549.0,0.588,2910,63,0.909,0.080
1997,57,F,2,High School,Divorced,Less than $40K,Blue,50,5,3,5,3276.0,0,3276.0,0.972,3758,61,0.694,0.000


In [ ]:
import io
from io import StringIO
csv_file = io.StringIO()
# by default sagemaker expects comma seperated
X.to_csv(csv_file, sep=",", header=False, index=False)
my_payload_as_csv = csv_file.getvalue()

In [ ]:
k = pd.read_csv("../df_test.csv")
import io
from io import StringIO
csv_file = io.StringIO()
# by default sagemaker expects comma seperated
k.to_csv(csv_file, sep=",", header=False, index=False)
my_payload_as_csv = csv_file.getvalue()

In [ ]:
payload = my_payload_as_csv.split('\n')  
payload

In [884]:
samples = []
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

for row in open('../df_test.csv','rb'):
    list1 = []
    row = str(row).replace("b'", "") 
    row = row.replace("\\n'", "") 
    row_l = row.split(',')
    #print(row_l)
    for i in row_l:
        if isint(i)==True:
            list1.append(int(i))
        elif isfloat(i)==True:
            list1.append(float(i))
        else:
            list1.append(i)
    samples.append(list1)

In [877]:
def input_fn(request_body,content_type):
    def isfloat(value):
      try:
        float(value)
        return True
      except ValueError:
        return False
    
    def isint(value):
      try:
        int(value)
        return True
      except ValueError:
        return False

    samples = []
    #data = request_body.split('\n')
    for row in request_body:
        list1 = []
        row = str(row).replace("b'", "") 
        row = row.replace("\\n'", "") 
        row_to_list = row.split(',')
        #print(row_to_list)
        for i in row_to_list:
            if isint(i)==True:
                list1.append(int(i))
            elif isfloat(i)==True:
                list1.append(float(i))
            else:
                list1.append(i)
        samples.append(list1)
    return samples

In [878]:
input_data = input_fn(open('../df_test.csv','rb'),'csv')

In [879]:
len(input_data)

2000

In [880]:
input_data[0]

[57,
 'M',
 3,
 'Unknown',
 'Single',
 '$120K +',
 'Silver',
 49,
 1,
 2,
 1,
 34516.0,
 2517,
 31999.0,
 0.693,
 8596,
 102,
 0.672,
 0.073]

In [886]:
df = pd.DataFrame(samples,columns=['Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio'])
df.dtypes

Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [882]:
df

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,57,M,3,Unknown,Single,$120K +,Silver,49,1,2,1,34516.0,2517,31999.0,0.693,8596,102,0.672,0.073
1,51,F,3,Uneducated,Married,Less than $40K,Blue,39,3,3,0,8148.0,701,7447.0,0.819,4124,87,0.891,0.086
2,55,F,2,Graduate,Married,Unknown,Blue,41,1,1,1,3591.0,1760,1831.0,0.507,13289,121,0.754,0.490
3,53,F,1,Graduate,Married,Less than $40K,Blue,32,4,2,3,7832.0,1784,6048.0,0.711,14614,106,0.656,0.228
4,43,F,5,Uneducated,Married,$40K - $60K,Blue,36,5,3,2,2311.0,1320,991.0,0.658,3967,71,0.690,0.571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,42,F,4,College,Married,$40K - $60K,Blue,34,1,3,2,1695.0,0,1695.0,0.453,2055,46,0.314,0.000
1996,49,F,2,Graduate,Single,Less than $40K,Blue,31,5,1,2,5456.0,951,4505.0,0.737,1096,27,0.588,0.174
1997,52,F,2,Graduate,Married,Less than $40K,Blue,43,5,2,2,8203.0,654,7549.0,0.588,2910,63,0.909,0.080
1998,57,F,2,High School,Divorced,Less than $40K,Blue,50,5,3,5,3276.0,0,3276.0,0.972,3758,61,0.694,0.000


In [ ]:
deploy_test = pd.read_csv("../df_test.csv").values.tolist()

deploy_test[0:2]

In [ ]:
df = pd.DataFrame(deploy_test,columns=['Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio'])
df